## Setup<a class="anchor" id="setup"></a>

In [2]:
%%capture
%pip install awswrangler

In [1]:
import os
import time
import boto3
import util
import sagemaker
import awswrangler as wr
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Data preparation

In [2]:
def loadRaw():
    bucket = ''
    path = ''
    bucket_uri = f's3://{bucket}/{path}'
    data_suffix = '.gz.parquet'
    cols = ['mt_sent_time', 'mt_category', 'operator', 'shortcode', 'mt_count', 'dn_count']
    df = wr.s3.read_parquet(path=bucket_uri, path_suffix=data_suffix, columns=cols, dataset=True)
    return df

In [3]:
raw = loadRaw()

In [ ]:
raw.tail()

# EDA


In [ ]:
raw.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

In [ ]:
raw.info()

In [ ]:
for col in raw.columns:
    print(col, raw[col].nunique())

In [ ]:
for col in raw.columns:
    print(col, raw[col].unique(), '\n')

In [ ]:
cols = ['mt_category', 'operator', 'shortcode']
for col in cols:
    print(raw[col].value_counts())

# Data preprocessing

In [12]:
def featureEngineering(raw, *args):
    # args = groupby list
    
    df = raw.copy()
    # Only these 2 categories are needed)
    mt_cat_list = ['schedule', 'retry1']
    df = df[df.mt_category.isin(mt_cat_list)]
    
    # Convert date/time from string to datetime format
    df['mt_sent_time'] = pd.to_datetime(df['mt_sent_time'], errors='coerce').dt.floor('H')
    
    # Group entries
    df = df.groupby(['mt_category', 'mt_sent_time', *args]).agg({'mt_count':'sum', 'dn_count':'sum'}).reset_index()
    # add scs_date
    df['scs_dn_rate'] = df.dn_count / df.mt_count
    
    # add columns describing different time frame
    df['mt_sent_time_hour'] = df['mt_sent_time'].dt.hour
    df['mt_sent_time_dayofweek'] = df['mt_sent_time'].dt.weekday
    # handle outliers with low support
    df.loc[df.mt_count<10, 'scs_dn_rate']=np.nan
    # handle outliers with extreme value
    # In this case, if scs rate >0.2 or <0.001, it is wrong, and replaced by null
    df.loc[(df.scs_dn_rate>0.2) | (df.scs_dn_rate<0.001), 'scs_dn_rate']=np.nan
    fill_max = lambda x: x.replace(1, x[x<1].max())   # Write function that Fills outliers in subset
    fill_min = lambda x: x.replace(0, x[x>0].min())
    df.loc[:,'scs_dn_rate'] = df.groupby(['mt_category', 'mt_sent_time_hour', 'mt_sent_time_dayofweek', *args])['scs_dn_rate'].apply(fill_max) 
    df.loc[:,'scs_dn_rate'] = df.groupby(['mt_category', 'mt_sent_time_hour', 'mt_sent_time_dayofweek', *args])['scs_dn_rate'].apply(fill_min)
    return df

In [13]:
print('null % of scs_dn_rate after preprocessing:')
df = featureEngineering(raw)
print('basic: ', df.scs_dn_rate.isna().sum()*100 / len(df))
df = featureEngineering(raw, 'operator')
print('operator: ', df.scs_dn_rate.isna().sum()*100 / len(df))
df = featureEngineering(raw, 'shortcode')
print('shortcode: ', df.scs_dn_rate.isna().sum()*100 / len(df))
df = featureEngineering(raw, 'operator', 'shortcode')
print('operator and shortcode: ', df.scs_dn_rate.isna().sum()*100 / len(df))

null % of scs_dn_rate after preprocessing:
basic:  17.24110796348757
operator:  20.68071491615181
shortcode:  29.685807150595885
operator and shortcode:  35.717744338498214


In [90]:
df = featureEngineering(raw)
df.mt_count.sum()

106190471

In [ ]:
df['mt_sent_time_date'] = df['mt_sent_time'].dt.date
print(df.groupby('mt_sent_time_date').mt_count.sum().describe())
df = df.drop(['mt_sent_time_date'], axis=1)

In [ ]:
df[['mt_count', 'dn_count', 'scs_dn_rate']].describe()

# Plot for mt_count

### average

In [ ]:
df.groupby(['mt_sent_time_hour', 'mt_category']).mt_count.mean().unstack().plot()

In [ ]:
df.groupby(['mt_sent_time_dayofweek', 'mt_category']).mt_count.mean().unstack().plot()

# Plot for scs_dn_rate

### timeseries

scs_dn_rate per hour

In [ ]:
df.tail()

In [106]:
def df_dayPeriod(raw, diff):
    df = raw.copy()
    df['mt_sent_time_date'] = df['mt_sent_time'].dt.date
    df['mt_sent_time_date'] = df['mt_sent_time_date'].astype(str)
    date = 'mt_sent_time_date'
    lastdate = df.mt_sent_time[-1:].item()
    start_date = str(lastdate - timedelta(days=diff))
    end_date = str(lastdate)
    mask = (df[date] > start_date) & (df[date] <= end_date)
    df=df.loc[mask]
    return df

## plot for past 7 days success rate

In [ ]:
plt.figure(figsize=(16,8))
sns.lineplot(x="mt_sent_time", y='scs_dn_rate', data=df_dayPeriod(df, 7))

## plot for lastday success rate

In [ ]:
plt.figure(figsize=(16,8))
sns.lineplot(x="mt_sent_time", y='scs_dn_rate', data=df_dayPeriod(df, 1))

per hour

In [ ]:
df.groupby(['mt_sent_time_hour', 'mt_category']).scs_dn_rate.mean().unstack().plot()

## last week

In [ ]:
df_dayPeriod(df, 7).groupby(['mt_sent_time_hour', 'mt_category']).scs_dn_rate.mean().unstack().plot()

per weekday

In [ ]:
df.groupby(['mt_sent_time_dayofweek', 'mt_category']).scs_dn_rate.mean().unstack().plot()

## for last week

In [ ]:
df_dayPeriod(df, 7).groupby(['mt_sent_time_dayofweek', 'mt_category']).scs_dn_rate.mean().unstack().plot()

## Plot for specific feature

In [133]:
def feature_plot(df, *args):
    fig1, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
    fig1.suptitle('schedule')
    axes[0].set_ylabel('mt_count')
    axes[1].set_ylabel('scs_dn_rate')
    df[df['mt_category']=='schedule'].groupby(['mt_sent_time_hour', *args]).mt_count.mean().unstack().fillna(0).plot(ax=axes[0])
    df[df['mt_category']=='schedule'].groupby(['mt_sent_time_hour', *args]).scs_dn_rate.mean().unstack().plot(ax=axes[1])
    fig2, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
    fig2.suptitle('retry1')
    axes[0].set_ylabel('mt_count')
    axes[1].set_ylabel('scs_dn_rate')
    df[df['mt_category']=='retry1'].groupby(['mt_sent_time_hour', *args]).mt_count.mean().unstack().fillna(0).plot(ax=axes[0])
    df[df['mt_category']=='retry1'].groupby(['mt_sent_time_hour', *args]).scs_dn_rate.mean().unstack().plot(ax=axes[1])

## Plot for day of week

In [ ]:
feature_plot(df, 'mt_sent_time_dayofweek')

## for last week

In [ ]:
feature_plot(df_dayPeriod(df,7), 'mt_sent_time_dayofweek')

## Plot for operator

In [135]:
df = featureEngineering(raw, 'operator')

In [ ]:
feature_plot(df, 'operator')

## for last week

In [ ]:
feature_plot(df_dayPeriod(df,7), 'operator')

## plot for shortcode

In [138]:
df = featureEngineering(raw, 'shortcode')

In [ ]:
feature_plot(df, 'shortcode')

## for last week

In [ ]:
feature_plot(df_dayPeriod(df,7), 'shortcode')

## Response feature engineering

In [ ]:
df.scs_dn_rate.value_counts()

## Creating XGBoost for feature importance

In [86]:
def create_features(df):

    # Only these 2 categories are needed
    mt_cat_list = ['schedule', 'retry1']
    df = df[df.mt_category.isin(mt_cat_list)]
    
    # Convert date/time from string to datetime format
    df.loc[:,'mt_sent_time'] = pd.to_datetime(df['mt_sent_time'], errors='coerce')
    df = df.groupby(['mt_sent_time', 'mt_category', 'operator', 'shortcode']).agg({'mt_count':'sum', 'dn_count':'sum'}).reset_index()
    
    # create time series features
    df['scs_dn_rate'] = df.dn_count / df.mt_count
    df['date'] = df.mt_sent_time
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    # handle outliers with low support
    df.loc[df.mt_count<5, 'scs_dn_rate']=np.nan
    # handle outliers with extreme value
    fill_max = lambda x: x.replace(1, x[x<1].max())   # Write function that Fills outliers in subset
    fill_min = lambda x: x.replace(0, x[x>0].min())
    fill_nan = lambda x: x.fillna(x.median())
    df.loc[:,'scs_dn_rate'] = df.groupby(['hour', 'dayofweek'])['scs_dn_rate'].apply(fill_max)  # Apply function to grouped column 
    df.loc[:,'scs_dn_rate'] = df.groupby(['hour', 'dayofweek'])['scs_dn_rate'].apply(fill_min)  # Apply function to grouped column
    df.loc[:,'scs_dn_rate'] = df.groupby(['hour', 'dayofweek'])['scs_dn_rate'].apply(fill_nan)  # Apply function to grouped column 
    
    # convert categorical data
    df = pd.get_dummies(df)

    X = df.drop(['scs_dn_rate', 'date', 'mt_sent_time', 'mt_count', 'dn_count'], axis=1)
    y = df['scs_dn_rate']
    return X, y

In [87]:
%%capture
%pip install xgboost

In [ ]:
import xgboost as xgb
X_train, y_train = create_features(raw)
reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train)],
        early_stopping_rounds=50, #stop if 50 consequent rounds without decrease of error
        verbose=False)

In [ ]:
xgb.plot_importance(reg, height=0.6)